# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,675.044782,-0.785241,1234,674.259542,675.830023,1348.519083,1351.660046
6,954.552587,0.592531,1234,955.145119,953.960056,1910.290237,1907.920112
7,189.491453,-0.011029,1234,189.480424,189.502481,378.960848,379.004963
8,741.254599,0.833940,1234,742.088539,740.420659,1484.177077,1480.841317
9,86.177330,-0.924920,1234,85.252411,87.102250,170.504821,174.204501
10,950.949035,-0.607123,1234,950.341913,951.556158,1900.683825,1903.112315
11,560.484166,0.716766,1234,561.200932,559.767400,1122.401863,1119.534800
12,124.389479,0.478123,1234,124.867602,123.911356,249.735203,247.822711
13,627.930534,-0.342141,1234,627.588393,628.272676,1255.176786,1256.545352
14,930.540616,0.482949,1234,931.023565,930.057667,1862.047130,1860.115334


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-674.259542,675.830023
6,-955.145119,953.960056
7,-189.480424,189.502481
8,-742.088539,740.420659
9,-85.252411,87.102250
10,-950.341913,951.556158
11,-561.200932,559.767400
12,-124.867602,123.911356
13,-627.588393,628.272676
14,-931.023565,930.057667


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-674.259542,675.830023
6,-955.145119,953.960056
7,-189.480424,189.502481
8,-742.088539,740.420659
9,-85.252411,87.102250
10,-950.341913,951.556158
11,-561.200932,559.767400
12,-124.867602,123.911356
13,-627.588393,628.272676
14,-931.023565,930.057667


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-674.25954157,  675.83002296],
       [-955.14511864,  953.96005608],
       [-189.48042417,  189.50248148],
       [-742.08853855,  740.42065871],
       [ -85.25241063,   87.10225028],
       [-950.34191255,  951.55615774],
       [-561.2009316 ,  559.7674    ],
       [-124.86760172,  123.91135552],
       [-627.58839287,  628.27267579],
       [-931.02356507,  930.0576672 ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1346.519083,-1351.660046
6,1908.290237,-1907.920112
7,376.960848,-379.004963
8,1482.177077,-1480.841317
9,168.504821,-174.204501
10,1898.683825,-1903.112315
11,1120.401863,-1119.534800
12,247.735203,-247.822711
13,1253.176786,-1256.545352
14,1860.047130,-1860.115334


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
